# Preprocessamento

Apesar do formato que as APIs da Riot entregam os dados serem excelentes, não dá pra usar do mesmo jeito que vem. A ideia desse notebooks é converter em CSVs e DataFrames pra facilitar a manipulação baseado em arquivo único. Pra fazer as primeiras visualizações e ter uma ideia dos dados, é bom dar separar os dados que vão ser utilizados. Também é aqui que devem ser criadas (ou estabelecidas) as ligações entre os dados das partidas e o que está no Dragontail.

In [1]:
import pandas as pd
import numpy as np
import altair as alt
import json
from pathlib import Path
from datetime import datetime
import time
from tqdm.notebook import trange, tqdm
import os
import pprint
from sys import getsizeof

In [2]:
ids = [match.split(".")[0] for match in os.listdir("matches/")]
with open("ODESAFIO.json","rb") as f: 
    nl_id = json.loads(f.read())["puuid"]

Para gravar tudo em um arquivo só, vamos separar as colunas que vem nos JSON, um conjunto de `info`, `participants` e `challenges` que vão ditar o perfil da partida. Como nosso interesse é o nicklink, então adicionar algumas coisas dele como participante diretamente.  
Eu adicionei também um outor objeto com a informação sobre a timeline, que é menos estruturada, então salvo o JSON direto e as posições separadas. Parece que muita coisa pode ser útil na visualização, e como a visualização das posições vai ser feitas em D3 de qualquer maneira, o JSON de informação parece bem útil.

In [3]:
challenge_keys = ["damagePerMinute","goldPerMinute","visionScorePerMinute","visionScoreAdvantageLaneOpponent","kda","maxCsAdvantageOnLaneOpponent","bountyGold"]
info_keys = ["gameDuration", "gameStartTimestamp"]
participant_keys = ["win","turretsLost", "bountyLevel", "damageDealtToTurrets", "damageDealtToObjectives", "damageDealtToBuildings","totalMinionsKilled"]

In [6]:
# partidas e linhas do tempo
matches_complete = []
for match_id in tqdm(ids[:]):
    with open(f'matches/{match_id}.json','r') as f:
        match = json.load(f)
    with open(f'timelines/{match_id}.json','r') as f:
        timeline = json.load(f)
    
    nl_pdto = [ p for p in match["info"]["participants"] if p["puuid"] == nl_id][0]
    del nl_pdto["perks"]
    match_data = {'id': match_id}
    tl_data = {'id': match_id}
    
    for key in info_keys:
        match_data[key] = match["info"][key]
    match_data["gameEndTimestamp"] = match["info"]["gameStartTimestamp"] + 1000 * match["info"]["gameDuration"]
    for key in participant_keys:
        match_data[key] = nl_pdto[key]
    for key in challenge_keys:
        if key in nl_pdto["challenges"]:
            match_data[key] = nl_pdto["challenges"][key]
        else:
            match_data[key] = np.nan
            
    tops = [ p for p in match["info"]["participants"] if p["teamPosition"] == "TOP"]
    match_data["champion"] = nl_pdto["championName"]
    match_data["laneOpponent"] = list(filter(lambda x: x["puuid"] != nl_id, tops))[0]["championName"]
    teamId = nl_pdto["teamId"]
    match_data["team"] = "blue" if teamId == 100 else "red"
    
    nl_ptid = str([ x for x in timeline["info"]["participants"] if nl_id == x["puuid"]][0]["participantId"])
    match_data["nlId"] = nl_ptid
    tl_data["nlId"] = nl_ptid
    
    frames = timeline["info"]["frames"]
    position = [[p["participantFrames"][nl_ptid]["position"]["x"],p["participantFrames"][nl_ptid]["position"]["y"]] for p in frames]
    events = [p["events"] for p in frames] 
    events = [e2 for e1 in events for e2 in e1]
    
    kills = [e for e in events if e["type"] == 'CHAMPION_KILL'] 
    kills = [e for e in kills  if e['killerId'] == int(nl_ptid)]
    
    deaths = [e for e in events if e["type"] == 'CHAMPION_KILL']
    deaths = [e for e in deaths if e['victimId'] == int(nl_ptid)]

    buildings = [e for e in events if e["type"] == 'BUILDING_KILL']
    buildings = [e for e in buildings if e['killerId'] == int(nl_ptid)]
    
    elite = [e for e in events if e["type"] == 'ELITE_MONSTER_KILL']
    elite = [e for e in elite if e['killerId'] == int(nl_ptid) or e['killerTeamId'] == teamId]

    fb = [e for e in events if e["type"] == 'CHAMPION_SPECIAL_KILL']
    fb = [e for e in fb if e['killerId'] == int(nl_ptid) and e["killType"] == "KILL_FIRST_BLOOD"]
    
    tl_data["position"] = [{"x": p[0], "y": p[1]} for p in position]
    tl_data["kills"] = kills
    tl_data["deaths"] = deaths
    tl_data["buildings"] = buildings
    tl_data["monster"] = elite
    
    matches_complete.append({'match': match_data, 'timeline': tl_data})        

  0%|          | 0/93 [00:00<?, ?it/s]

In [8]:
match_data = pd.DataFrame([match["match"] for match in matches_complete])
match_data

,id,gameDuration,gameStartTimestamp,gameEndTimestamp,win,turretsLost,bountyLevel,damageDealtToTurrets,damageDealtToObjectives,damageDealtToBuildings,...,goldPerMinute,visionScorePerMinute,visionScoreAdvantageLaneOpponent,kda,maxCsAdvantageOnLaneOpponent,bountyGold,champion,laneOpponent,team,nlId
0,BR1_2479360102,942,1647024087174,1647025029174,True,0,2,10372,18693,10372,...,540.648721,0.290858,2.670087,3.000000,98.00,0,Jax,Caitlyn,blue,1
1,BR1_2479368408,1413,1647031280465,1647032693465,True,0,1,15783,24783,15783,...,651.097720,0.424057,0.796724,5.600000,69.00,0,Jax,Sett,red,6
2,BR1_2479381005,859,1647025630447,1647026489447,True,0,7,5130,14835,5130,...,582.336597,0.489374,1.003789,8.000000,48.00,0,Jax,Kayle,red,6
3,BR1_2479394996,1585,1647029182937,1647030767937,True,2,6,10022,13380,10022,...,649.008321,0.641166,1.446039,5.000000,99.00,550,Tryndamere,Jayce,blue,1
4,BR1_2479402377,1384,1647026985744,1647028369744,True,0,2,12106,20184,12106,...,641.041324,0.242456,0.267924,21.000000,92.50,0,Jax,Gnar,blue,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,BR1_2481017412,1389,1647230554433,1647231943433,True,0,0,2394,6473,2394,...,350.341024,0.541106,1.121057,2.000000,39.00,0,Irelia,Kennen,blue,1
89,BR1_2481020856,1696,1647233246409,1647234942409,True,3,8,8278,13142,8278,...,543.644755,0.670119,0.620147,6.000000,50.25,500,Jax,Mordekaiser,red,6
90,BR1_2481041619,1615,1647237009921,1647238624921,True,3,6,9977,13525,9977,...,437.485678,0.373521,NaN,15.000000,NaN,300,Jax,Malphite,red,6
91,BR1_2481060618,1121,1647235318821,1647236439821,True,3,3,1456,1604,1456,...,299.106265,0.543786,1.044728,1.000000,2.00,150,Jax,Jayce,blue,1


In [11]:
timeline_data = pd.DataFrame([match["timeline"] for match in matches_complete])
timeline_data

,id,nlId,position,kills,deaths,buildings,monster
0,BR1_2479360102,1,"[{'x': 554, 'y': 581}, {'x': 3722, 'y': 9502},...","[{'bounty': 400, 'killStreakLength': 0, 'kille...","[{'bounty': 300, 'killStreakLength': 0, 'kille...","[{'bounty': 0, 'buildingType': 'TOWER_BUILDING...","[{'bounty': 0, 'killerId': 1, 'killerTeamId': ..."
1,BR1_2479368408,6,"[{'x': 14180, 'y': 14271}, {'x': 13328, 'y': 1...","[{'assistingParticipantIds': [8, 10], 'bounty'...","[{'assistingParticipantIds': [1], 'bounty': 30...","[{'bounty': 0, 'buildingType': 'TOWER_BUILDING...","[{'assistingParticipantIds': [8], 'bounty': 0,..."
2,BR1_2479381005,6,"[{'x': 14180, 'y': 14271}, {'x': 4435, 'y': 11...","[{'bounty': 300, 'killStreakLength': 0, 'kille...",[],"[{'bounty': 0, 'buildingType': 'TOWER_BUILDING...","[{'bounty': 0, 'killerId': 7, 'killerTeamId': ..."
3,BR1_2479394996,1,"[{'x': 554, 'y': 581}, {'x': 898, 'y': 4470}, ...","[{'bounty': 400, 'killStreakLength': 0, 'kille...","[{'bounty': 300, 'killStreakLength': 0, 'kille...","[{'bounty': 0, 'buildingType': 'TOWER_BUILDING...","[{'bounty': 0, 'killerId': 2, 'killerTeamId': ..."
4,BR1_2479402377,1,"[{'x': 554, 'y': 581}, {'x': 5109, 'y': 8925},...","[{'bounty': 274, 'killStreakLength': 0, 'kille...","[{'assistingParticipantIds': [7], 'bounty': 30...","[{'bounty': 0, 'buildingType': 'TOWER_BUILDING...","[{'assistingParticipantIds': [3], 'bounty': 0,..."
...,...,...,...,...,...,...,...
88,BR1_2481017412,1,"[{'x': 554, 'y': 581}, {'x': 3723, 'y': 9529},...","[{'assistingParticipantIds': [2], 'bounty': 27...","[{'assistingParticipantIds': [6], 'bounty': 40...",[],"[{'bounty': 0, 'killerId': 2, 'killerTeamId': ..."
89,BR1_2481020856,6,"[{'x': 14180, 'y': 14271}, {'x': 4441, 'y': 11...","[{'bounty': 274, 'killStreakLength': 0, 'kille...","[{'assistingParticipantIds': [3, 5], 'bounty':...","[{'bounty': 0, 'buildingType': 'TOWER_BUILDING...","[{'assistingParticipantIds': [10], 'bounty': 0..."
90,BR1_2481041619,6,"[{'x': 14180, 'y': 14271}, {'x': 11821, 'y': 3...","[{'assistingParticipantIds': [7], 'bounty': 30...","[{'assistingParticipantIds': [4, 5], 'bounty':...","[{'bounty': 0, 'buildingType': 'TOWER_BUILDING...","[{'assistingParticipantIds': [10], 'bounty': 0..."
91,BR1_2481060618,1,"[{'x': 554, 'y': 581}, {'x': 3719, 'y': 9428},...","[{'assistingParticipantIds': [4, 5], 'bounty':...","[{'bounty': 400, 'killStreakLength': 0, 'kille...",[],"[{'assistingParticipantIds': [5], 'bounty': 0,..."


In [7]:
#match_data[match_data["win"] == True]["win"].count()

In [9]:
# len(matches_complete)

In [10]:
# getsizeof(match_data)

In [12]:
match_data.to_csv("match_data.csv", index=False)
timeline_data.to_csv("timeline_data.csv", index=False)